In this tutorial, we will demonstrate how DFT architectures can be created and simulated in Python. We are going to use two libraries:
- [DynamicFieldPy](https://github.com/danielsabinasz/DynamicFieldPy) (DFPy): A high-level API that allows to define DFT architectures in a generic way, i.e., in a way that is uncommitted as to the means by which the architecture is simulated (e.g., on a CPU, a GPU, or neuromorphic hardware).
- [DynamicFieldFlow](https://github.com/danielsabinasz/DynamicFieldFlow) (DFF): A library that allows to simulate DFPy architectures by means of the TensorFlow framework (https://www.tensorflow.org)